In [2]:
# BASIC IMPORTS OF ALL LIBRARIES
'''
Glove Imports are 300-dim from GENSIM
Pickle saves all the datasets
'''
import pygraphviz
from nltk import pos_tag
import csv
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
from copy import deepcopy
import pickle
from entity_groups import *
from graphviz import Source
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
import gensim.downloader as api

wnl = WordNetLemmatizer()
word_vectors = api.load("glove-wiki-gigaword-300")

In [3]:
# junk verbs that are generalizably illegal
junk_rel = ["be","have","do","let","","make","tell","say","describe","decide","want","name","know","think","try","become","oneday","put","come",'see', 'need', 'look', 'help', 'come', 'take', 'get', 'put', 'pick', 'turn', 'go', 'stand', 'give', 'notice', 'use',"get","start"]

# hypothetical words that are generally useless / qualifiers
junk_words = ["would","could","should","maybe","perhaps","think","might","assume","claim","this"]

# negative sentences
junk_words.extend(['n\'t',"not"])

# perspective actants that are pointless
junk = ["you","i","we","the","it","he","she","steinbeck","people","author",
        "book","me","steinback","him","her","their","this","","shelley","mary","harper","lee","tolkien"]

In [4]:
DATE = "0417"
file_name = "GDELT/"
text = "GDELT"

In [8]:
import pandas as pd
entity_file = "../Covid_19_Data/Extracted_Data/OS_News/" + "df_ent_final_ranking.csv"
S_use = {}
# fill this in

places = []
ent_file = pd.read_csv(entity_file)
for row in ent_file.head(n=20).iterrows():
    if row[1]['type'] == "GPE":
        places.append(row[1]['entity'])
    S_use[row[1]['entity']] = row[1]['entity']
    
actants_5g = ["5g","radio","transmitters","towers","radiation","army","waves","conspiracy"]
for a in actants_5g:
    S_use[a] = a

In [14]:
path = "Extracted_0416/"
date_start = "20200101000001"
date_end = "20200410000001"
date = date_start
results = "Results0417"

In [15]:
def extract_traj(file_name_re):
    trajectories = []
    same_post = 0
    traj = ["START"]

    with open(file_name_re) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')

        for row in readCSV:
            FLAG = 0
            sentence_num = row[1]
            sub_r = ""
            obj_r = ""
            sub_verb = ""

            if hypothetical_sentence(row[2]):
                continue

            try:
                sub = re.search(r"\{(\w+)\}", row[3]).group(1).lower()
                rel = wnl.lemmatize(re.search(r"\{(\w+)\}", row[4]).group(1).lower(), pos='v')
                obj = re.search(r"\{(\w+)\}", row[5]).group(1).lower()
            except:
                continue

            words = row[2].split()
            rel_words = []

            if rel == None or obj == None or sub == None:
                continue
            
            if same_post == row[0]:
                pass
            else:
                same_post = row[0]
                traj.append("TERMINATE")
                trajectories.append(traj)
                traj = ["START"]

            try:
                sub_r = S_use[sub]
            except:
                sub_r = sub
                FLAG += 1
            
            try:
                obj_r = S_use[obj]
            except:
                obj_r = obj
                FLAG += 1

            sub_verb = ""

            try:
                sub_verb = C_use[rel]
            except:
                sub_verb = rel

            if sub_r == obj_r or sub_r in junk or obj_r in junk or sub_verb in junk_rel \
                                    or FLAG > 0 or sub_r.isdigit() or obj_r.isdigit():
                continue

            traj.append(sub_verb)
            traj.append(sub_r + "_" + obj_r + "_" + sentence_num)

    # if we missed out on last post
    if traj != "START":
        traj.append("TERMINATE")
        trajectories.append(traj)

In [16]:
def find_mat(trajectories):
    rev_dict = {}
    precedence_matrix = np.zeros((50000,50000))
    dictionary_of_labels = {"START":0, "TERMINATE":1}
    counter_of_labels = defaultdict(int)
    max_rows = 1
    max_cols = 1
    counter = 2

    for trajectory in trajectories:

        triplets = ["START"]
        index = 1

        while index < len(trajectory)-2:
            triplets.append(trajectory[index] + '_' + trajectory[index + 1])
            index += 2

        triplets.append("TERMINATE")

        for key,_ in enumerate(triplets[1:-1]):
            removed_triplets = triplets[key+1].split("_")
            if len(removed_triplets) > 1:
                subject_ = removed_triplets[1]
                object_ = removed_triplets[2]
                if subject_ not in dictionary_of_labels:
                    dictionary_of_labels[subject_] = counter
                    counter += 1
                if object_ not in dictionary_of_labels:
                    dictionary_of_labels[object_] = counter
                    counter += 1
        
        triplet_temp = []

        # find uniques
        for trip in triplets:
            try:
                removed_triplet = trip.rsplit("_",1)[0]
            except:
                removed_triplet = trip

            if removed_triplet not in [t.rsplit("_",1)[0] for t in triplet_temp]:
                triplet_temp.append(trip)

        triplets = triplet_temp

        # for trip in triplets:
        #     counter_of_labels[trip] += 1

        alpha = 1.0

        for key in range(1,len(triplets)):
            removed_triplets = triplets[key].split("_")
            try:
                subject_ = removed_triplets[1]
                object_ = removed_triplets[2]
                precedence_matrix[dictionary_of_labels[subject_]]\
                                            [dictionary_of_labels[object_]] += 1.0
            except:
                pass


    precedence_matrix = precedence_matrix[:counter,:counter]
    for key in dictionary_of_labels:
        rev_dict[dictionary_of_labels[key]] = key

    return precedence_matrix, dictionary_of_labels, rev_dict

In [18]:
def plot_fig(trajectories,label):    
    while i < len(trajectories):
        sub_t = trajectories
        mat, dic, rev_dict = find_mat(sub_t)

        G = nx.from_numpy_matrix(mat, create_using=nx.DiGraph)
        DISP_THRESH = 2

        bb = {}
        for index, val in np.ndenumerate(mat):
            if val > 1:
                bb[index] = {"color":"red", "penwidth":2}
            else:
                bb[index] = {"color":"blue"}
        nx.set_edge_attributes(G, bb)

        bb1 = {}
        for j in range(mat.shape[0]):
            if rev_dict[j] in actants_5g:
                bb1[j] = {"color":"red", "penwidth":4}
            else:
                bb1[j] = {"color":"blue"}

        nx.set_node_attributes(G, bb1)

        bb2 = {}
        for j in range(mat.shape[0]):
            if rev_dict[j] in places:
                bb2[j] = {"color":"green", "penwidth":4}
            else:
                bb2[j] = {"color":"blue"}
        nx.set_node_attributes(G, bb2)

        H = nx.relabel_nodes(G, rev_dict)
        H.remove_nodes_from(list(nx.isolates(H)))
        A = nx.nx_agraph.to_agraph(H)

        A.layout('dot')
        A.draw(results + "Actant_TimeSeries/step" + str(label) + '.png')

In [19]:
from datetime import datetime, timedelta
def to_date(DATE):
    return datetime.strptime(DATE, '%Y%m%d%H%M%S')

def from_date(DATE):
    return DATE.strftime("%Y%m%d%H%M%S")

def create_range(DATE):
    list_dates = []
    for i in range(5):
        list_dates.append(DATE + timedelta(days = i))
    return list_dates

In [20]:
date_real = to_date(date)
date_end_real = to_date(date_end)

while date_real <= date_end_real:    
    trajs = []
    date_margin = create_range(date_real)

    for d in date_margin:
        ner_file = path + from_date(d) + "/into_relex_final_relations_-1.csv"
        trajs.extend(extract_traj(ner_file))

    plot_fig(traj, from_date(date_real))
    date_real += timedelta(days = 1)

FileNotFoundError: [Errno 2] No such file or directory: 'Extracted_0416/20200101000001into_relex_final_relations_-1.csv'